# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Save api_key information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}"
print(query_url) 


In [37]:
#Create lists for dataframe and API Calls 
city_name = []
country = []
clouds = []
lat = []
lngs = []
temp = []
max_temp = []
min_temp = []
humidity = []
wind = []

#Keep count
city_record = 1

#Statement of API log 
print("Data Retreieved:")
print("---------------------")

#Loop through cities in lists:
for city in cities:
    try:
        response = requests.get(f"{query_url}&q={city}").json()
        city_name.append(response["name"])
        country.append(response["sys"]["country"])
        clouds.append(response['clouds']['all'])
        lat.append(response["coord"]["lat"])
        lngs.append(response["coord"]["lon"])
        temp.append(response["main"]["temp"])
        max_temp.append(response["main"]["temp_max"])
        min_temp.append(response["main"]["temp_min"])
        humidity.append(response["main"]["humidity"])
        wind.append(response["wind"]["speed"])

        print(f"Displaying Record #{city_record} for {city}")

        #Increase counter
        city_record += 1

    #If no city found, handle for exception:
    except Exception:
        print("City not found, next city...")
        pass

Data Retreieved:
---------------------
Displaying Record 1 for punta arenas
Displaying Record 2 for rundu
Displaying Record 3 for qaanaaq
Displaying Record 4 for cherskiy
Displaying Record 5 for busselton
Displaying Record 6 for bredasdorp
Displaying Record 7 for bengkulu
Displaying Record 8 for barrow
Displaying Record 9 for seoul
City not found, next city...
Displaying Record 10 for muisne
Displaying Record 11 for cape town
Displaying Record 12 for port-gentil
Displaying Record 13 for coos bay
Displaying Record 14 for filadelfia
Displaying Record 15 for hobart
Displaying Record 16 for lorengau
City not found, next city...
Displaying Record 17 for rikitea
Displaying Record 18 for cidreira
Displaying Record 19 for troitskoye
Displaying Record 20 for pampa
Displaying Record 21 for bethel
Displaying Record 22 for clyde river
Displaying Record 23 for atuona
Displaying Record 24 for albany
Displaying Record 25 for bonfim
Displaying Record 26 for touros
Displaying Record 27 for husavik
Disp

KeyboardInterrupt: 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [38]:
#Convert data to DataFrame:
weather_data = {
    "City": city_name,
    "Country": country,
    "Latitude": lat,
    "Longitude": lngs,
    "Cloudiness": clouds,
    "Temperature": temp,
    "Max Temp": max_temp,
    "Min Temp": min_temp,
    "Humidity": humidity,
    "Wind Speed": wind
}

weather_df = pd.DataFrame(weather_data)
weather_df

#Export Dataframe to CSV
weather_df.to_csv("weather_data")

,City,Country,Latitude,Longitude,Cloudiness,Temperature,Max Temp,Min Temp,Humidity,Wind Speed
0,Punta Arenas,CL,-53.1500,-70.9167,36,37.11,37.11,37.11,87,29.08
1,Rundu,NA,-17.9333,19.7667,5,56.26,56.26,56.26,48,6.08
2,Qaanaaq,GL,77.4840,-69.3632,85,35.22,35.22,35.22,79,6.76
3,Cherskiy,RU,68.7500,161.3000,95,67.87,67.87,67.87,43,12.50
4,Busselton,AU,-33.6500,115.3333,28,59.02,59.02,59.02,56,4.29
...,...,...,...,...,...,...,...,...,...,...
515,Mezen',RU,65.8522,44.2400,100,48.99,48.99,48.99,79,4.81
516,Tauragė,LT,55.2522,22.2897,94,65.12,65.12,65.12,94,3.87
517,Karaton,ID,-6.3424,106.1045,100,84.15,84.15,84.15,66,2.55
518,Nuuk,GL,64.1835,-51.7216,100,43.41,43.41,41.88,93,25.32


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression